In [1]:
import os, glob
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from zipfile import ZipFile
import argparse
import pdb
import HEI
import sys

# Dictionaries and lists

In [2]:
important=['Participant ID','VEG0100','VEG0200','VEG0300','VEG0400','VEG0800','VEG0450','VEG0700',
       'VEG0600','VEG0900','VEG0500','VEG0100','VEG0700','FRU0100','FRU0200','FRU0300','FRU0400',
       'FRU0500','FRU0600','FRU0700','FRU0300','FRU0400','FRU0500','FRU0600','FRU0700',
       'Whole Grains (ounce equivalents)','DMF0100','DMR0100','DML0100','DMN0100','DMF0200',
       'DMR0200','DML0200','DML0300','DML0400','DCF0100','DCR0100','DCL0100','DCN0100','DYF0100',
       'DYR0100','DYL0100','DYF0200','DYR0200','DYL0200','DYN0100','DOT0300','DOT0400','DOT0500',
       'DOT0600','DOT0100','MRF0100','MRL0100','MRF0200','MRL0200','MRF0300','MRL0300','MRF0400',
       'MRL0400','MCF0200','MCL0200','MRF0500','MPF0100','MPL0100','MPF0200','MFF0100','MFL0100',
       'MFF0200','MSL0100','MSF0100','MCF0100','MCL0100','MOF0100','MOF0200','MOF0300','MOF0400',
       'MOF0500','MOF0600','MOF0700','VEG0700','MFF0100','MFL0100','MFF0200','MSL0100','MSF0100',
       'MOF0500','MOF0600','MOF0700','VEG0700','Sodium (mg)','Refined Grains (ounce equivalents)',
       'Added Sugars (by Total Sugars) (g)','% Calories from SFA','Energy (kcal)',
       'Total Polyunsaturated Fatty Acids (PUFA) (g)','Total Monounsaturated Fatty Acids (MUFA) (g)',
       'Total Saturated Fatty Acids (SFA) (g)','FRU0100','FRU0200','DMF0200','DMR0200','DML0200','DML0300',
       'SWT0600','BVS0400','BVS0300','BVS0500','BVS0100','BVS0200','BVS0600','BVS0700', 'SWT0600','SWT0100',
       'SWT0200','SWT0300','SWT0700' , 'SWT0800','SWT0400','SWT0500','DOT0300' , 'DOT0400',
       'DOT0100','DOT0200','GRR0800' , 'GRS0800',  'GRW0800','GRW0900' ,'GRS0900' , 'GRR0900' ,
       'GRW1100' , 'GRW1200', 'GRW0400' ,'GRS0400' ,'GRR0400','VEG0800','FMC0100', 'FMC0200','DMF0100',
       'DMR0100','DML0100','DMN0100' ,'DMF0200','DMR0200','DML0200','DOT0500', 'DOT0600','DOT0700',
       'DOT0800','GRW0600','GRS0600','GRR0600','GRW0700','GRS0700','GRR0700','GRW1300','GRS1300']

para_dict = {
        'hei_totveg': {'parameters':[1.1], 'name': 'HEIX1_TOTALVEG'},
        'hei_greensbeans': {'parameters':[0.2], 'name': 'HEIX2_GREEN_AND_BEAN'},
        'hei_totfruit': {'parameters':[0.8], 'name': 'HEIX3_TOTALFRUIT'},
        'hei_wholefruit': {'parameters':[0.4], 'name': 'HEIX4_WHOLEFRUIT'},
        'hei_wholegrains': {'parameters':[1.5], 'name': 'HEIX5_WHOLEGRAIN'},
        'hei_dairy': {'parameters':[1.3], 'name': 'HEIX6_TOTALDAIRY'},
        'hei_totproteins': {'parameters':[2.5], 'name': 'HEIX7_TOTPROT'},
        'hei_seafoodplantprot': {'parameters':[0.8], 'name': 'HEIX8_SEAPLANT_PROT'},
        'hei_refinedgrains': {'parameters':[1.8,4.3], 'name': 'HEIX11_REFINEDGRAIN'},
        'hei_addedsugars': {'parameters':[6.5,26], 'name': 'HEIX12_ADDEDSUGARS'},
        'hei_SFA': {'parameters':[8,16], 'name': 'HEIX13_SATFATS'},
        'Fats': {'parameters':[1.2,2.5], 'name': 'HEIX9_FATTYACID'},
        'hei_sodium':{'parameters':[1.1,2.0],'name':'HEIX10_SODIUM'}
        }

ped811_dict = {
    'hei_vegetables': {'parameters':[0.1,1.9], 'name': 'HEIX1_VEGETABLES'},
    'hei_totfruit': {'parameters':[0.1,1.9], 'name': 'HEIX2_TOTALFRUIT'},
    'hei_wholegrains': {'parameters':[1.0,3.5,0.0 , 8.0], 'name': 'HEIX3_WHOLEGRAIN'},
    'hei_milk': {'parameters':[20,28,8,35], 'name': 'HEIX4_TOTALDAIRY'},
    'hei_proteins': {'parameters':[2.5,6.0,0,10.0], 'name': 'HEIX5_PROTEIN'},
    'hei_refinedgrains': {'parameters':[1.6,3.5], 'name': 'HEIX6_REFINEDGRAIN'},
    'hei_fruitjuice': {'parameters':[0.1,6.0], 'name': 'HEIX7_FRUITJUICE'},
    'hei_SSB': {'parameters':[0.1,4.0], 'name': 'HEIX8_SSB'},
    'hei_sweets': {'parameters':[0.1,1.0], 'name': 'HEIX9_SWEETS'},
    'hei_salty': {'parameters':[0.1,1.0], 'name': 'HEIX10_SALTY'}
        }

ped1224_dict ={
    'hei_vegetables': {'parameters':[0.1,7.9], 'name': 'HEIX1_VEGETABLES'},
    'hei_totfruit': {'parameters':[0.1,7.9], 'name': 'HEIX2_TOTALFRUIT'},
    'hei_wholegrains': {'parameters':[1.5, 5.5, 0, 8.0], 'name': 'HEIX3_WHOLEGRAIN'},
    'hei_milk': {'parameters':[14.0,18.0,8.0,24.0], 'name': 'HEIX4_TOTALDAIRY'},
    'hei_proteins': {'parameters':[2.0,3.0,0,6.0], 'name': 'HEIX5_PROTEIN'},
    'hei_refinedgrains': {'parameters':[1.9, 4.2], 'name': 'HEIX6_REFINEDGRAIN'},
    'hei_fruitjuice': {'parameters':[4.1,6.0], 'name': 'HEIX7_FRUITJUICE'},
    'hei_SSB': {'parameters':[0.1,4.0], 'name': 'HEIX8_SSB'},
    'hei_sweets': {'parameters':[0.1,1.0], 'name': 'HEIX9_SWEETS'},
    'hei_salty': {'parameters':[0.1,1.0], 'name': 'HEIX10_SALTY'}
        }

ped08_dict = {
    'hei_vegetables': {'parameters':[0,0], 'name': 'HEIX1_VEGETABLES'},
    'hei_totfruit': {'parameters':[0,0], 'name': 'HEIX2_TOTALFRUIT'},
    'hei_wholegrains': {'parameters':[0,0,0,0], 'name': 'HEIX3_WHOLEGRAIN'},
    'hei_dairy': {'parameters':[0,0,0,0], 'name': 'HEIX4_TOTALDAIRY'},
    'hei_proteins': {'parameters':[0,0,0,0], 'name': 'HEIX5_PROTEIN'},
    'hei_refinedgrains': {'parameters':[0,0], 'name': 'HEIX6_REFINEDGRAIN'},
    'hei_fruitjuice': {'parameters':[0,0], 'name': 'HEIX7_FRUITJUICE'},
    'hei_SSB': {'parameters':[0,0], 'name': 'HEIX8_SSB'},
    'hei_sweets': {'parameters':[0,0], 'name': 'HEIX9_SWEETS'},
    'hei_salty': {'parameters':[0,0], 'name': 'HEIX10_SALTY'},
    'hei_cereal': {'parameters':[0,0], 'name': 'HEIX11_CEREAL'}
    }

ped_dict={
    'young': ped811_dict,
    'child':ped1224_dict,
    'infant':ped08_dict
}


hei_dict={
      'hei_totveg':
      ['VEG0100','VEG0200','VEG0300','VEG0400','VEG0800','VEG0450','VEG0700','VEG0600','VEG0900','VEG0500'],
      'hei_greensbeans':
      ['VEG0100','VEG0700'],
      'hei_totfruit':
      ['FRU0100','FRU0200','FRU0300','FRU0400','FRU0500','FRU0600','FRU0700'],
      'hei_wholefruit':
      ['FRU0300','FRU0400','FRU0500','FRU0600','FRU0700'],
      'hei_wholegrains':
      ['Whole Grains (ounce equivalents)'],
      'hei_dairy':
      ['DMF0100','DMR0100','DML0100','DMN0100','DMF0200','DMR0200','DML0200',
                   'DML0300','DML0400','DCF0100','DCR0100','DCL0100','DCN0100','DYF0100',
                   'DYR0100','DYL0100','DYF0200','DYR0200','DYL0200','DYN0100',
                   'DOT0300','DOT0400','DOT0500','DOT0600','DOT0100'],
      'hei_totproteins':
      ['MRF0100','MRL0100','MRF0200','MRL0200','MRF0300','MRL0300','MRF0400',
                         'MRL0400','MCF0200','MCL0200','MRF0500','MPF0100','MPL0100','MPF0200',
                         'MFF0100','MFL0100','MFF0200','MSL0100',
                         'MSF0100','MCF0100','MCL0100','MOF0100','MOF0200','MOF0300','MOF0400','MOF0500',
                         'MOF0600','MOF0700','VEG0700'],
      'hei_seafoodplantprot':
      ['MFF0100','MFL0100','MFF0200','MSL0100','MSF0100','MOF0500','MOF0600','MOF0700','VEG0700'],
      'hei_sodium':
      ['Sodium (mg)'],
      'hei_refinedgrains':
      ['Refined Grains (ounce equivalents)'],
      'hei_addedsugars':
      ['Added Sugars (by Total Sugars) (g)'],
      'ripctsfa': ['% Calories from SFA','Energy (kcal)'],
     'energy':
     ['Energy (kcal)'],
     'fats':
     ['Total Polyunsaturated Fatty Acids (PUFA) (g)','Total Monounsaturated Fatty Acids (MUFA) (g)',
     'Total Saturated Fatty Acids (SFA) (g)']
     }

hei_ped_dict={'hei_totveg':
        ['VEG0100','VEG0200','VEG0300','VEG0400','VEG0800','VEG0450','VEG0700','VEG0600','VEG0900','VEG0500'],
        'hei_greensbeans':
        ['VEG0100','VEG0700'],
        'hei_totfruit':
        ['FRU0100','FRU0200','FRU0300','FRU0400','FRU0500','FRU0600','FRU0700'],
        'hei_wholefruit':
        ['FRU0300','FRU0400','FRU0500','FRU0600','FRU0700'],
        'hei_wholegrains':
        ['Whole Grains (ounce equivalents)'],
        'hei_dairy':
        ['DMF0100','DMR0100','DML0100','DMN0100','DMF0200','DMR0200','DML0200',
                     'DML0300','DML0400','DCF0100','DCR0100','DCL0100','DCN0100','DYF0100',
                     'DYR0100','DYL0100','DYF0200','DYR0200','DYL0200','DYN0100',
                     'DOT0300','DOT0400','DOT0500','DOT0600','DOT0100'],
        'hei_totproteins':
        ['MRF0100','MRL0100','MRF0200','MRL0200','MRF0300','MRL0300','MRF0400',
                           'MRL0400','MCF0200','MCL0200','MRF0500','MPF0100','MPL0100','MPF0200',
                           'MFF0100','MFL0100','MFF0200','MSL0100',
                           'MSF0100','MCF0100','MCL0100','MOF0100','MOF0200','MOF0300','MOF0400','MOF0500',
                           'MOF0600','MOF0700','VEG0700'],
        'hei_seafoodplantprot':
        ['MFF0100','MFL0100','MFF0200','MSL0100','MSF0100','MOF0500','MOF0600','MOF0700','VEG0700'],
        'hei_refinedgrains':
        ['Refined Grains (ounce equivalents)'],
       'energy':
       ['Energy (kcal)'],
        'hei_fruitjuice':
        ['FRU0100','FRU0200'],
        'hei_SSB':
        ['DMF0200','DMR0200','DML0200','DML0300','SWT0600','BVS0400','BVS0300','BVS0500','BVS0100','BVS0200','BVS0600','BVS0700', 'SWT0600'],
        'chocolate_candies':
        ['SWT0100'],
        'candies':
        ['SWT0200'],
        'frosting':
        ['SWT0300'],
        'sweet_sauce':
        ['SWT0700' , 'SWT0800' ],
        'sugar':
        ['SWT0400' ],
        'syrups':
        ['SWT0500' ],
        'Pudding':
        ['DOT0300' , 'DOT0400'],
        'icecream':
        ['DOT0100' ],
        'nondairy_treat':
        ['DOT0200' ],
        'baked_good':
        ['GRR0800' , 'GRS0800',  'GRW0800' ],
        'chips':
        ['GRW0900' ,'GRS0900' , 'GRR0900' , 'GRW1100' , 'GRW1200', 'GRW0400' ,'GRS0400' ,'GRR0400' ],
        'fries':
        ['VEG0800' ],
        'other_fried':
        ['FMC0100', 'FMC0200' ],
        'milk':
        ['DMF0100','DMR0100','DML0100','DMN0100' ,'DMF0200','DMR0200','DML0200','DOT0500', 'DOT0600'],
        'formula_foz':
        ['DOT0700', 'DOT0800'],
        'cereal_oz':
        ['GRW0600','GRS0600','GRR0600','GRW0700','GRS0700','GRR0700'],
        'bbcereal_hcup':
        ['GRW1300','GRS1300']
        }

conv_dict={
    'hei_fruitjuice':4,
    'hei_SSB':8,
    'chocolate_candies':40,
    'candies':15,
    'frosting':35,
    'sugar':4,
    'syrups':.25,
    'Pudding':1,
    'icecream':.5,
    'nondairy_treat':85,
    'baked_good':55,
    'fries': 70,
    'formula_foz':5,
    'bbcereal_hcup':.5
}

make_hei_dict={
    'hei_sweets' :
    ['chocolate_candies','candies','frosting','sweet_sauce','sugar','syrups','Pudding', 'icecream','nondairy_treat','baked_good'],
    'hei_salty' :
    ['chips','other_fried','fries'],
    'hei_vegetables':
    ['hei_totveg', 'hei_greensbeans'],
    'hei_proteins':
    ['hei_totproteins','hei_seafoodplantprot'],
    'hei_milk':
    ['hei_dairy','formula_foz'],
    'hei_cereal':
    ['cereal_oz', 'bbcereal_hcup']
}

interest = ['Participant ID','Energy (kcal)', 'hei_totveg', 'hei_greensbeans', 'hei_totfruit', 'hei_wholefruit', 'hei_wholegrains','hei_dairy', 'hei_totproteins', 'hei_seafoodplantprot', 'Total Polyunsaturated Fatty Acids (PUFA) (g)',
        'Total Monounsaturated Fatty Acids (MUFA) (g)', 'Total Saturated Fatty Acids (SFA) (g)',
        'hei_sodium', 'hei_refinedgrains', 'hei_addedsugars', 'ripctsfa','energy','% Calories from SFA']

ped_interest = ['Participant ID','Energy (kcal)', 'hei_totveg', 'hei_greensbeans', 'hei_totfruit', 'hei_wholefruit', 'hei_wholegrains','hei_dairy', 'hei_totproteins', 'hei_seafoodplantprot',
        'hei_refinedgrains', 'hei_addedsugars', 'ripctsfa','energy','chocolate_candies','candies','frosting','sweet_sauce','sugar','syrups','Pudding', 'icecream','nondairy_treat','baked_good',
        'chips','other_fried','fries']


In [3]:
basepath='/Users/gracer/Google Drive/BCP/data'

In [27]:
arglist={'BASEPATH': basepath, 
         'CHILD': True, 
         'NAMES': 'testies', 
         'OPTS': ['Chil'], 
         'SAVE':os.path.join(basepath, 'derivived'),
         'XTRA':'total_dataset.csv'}

In [5]:
for (dirpath, dirnames, filenames) in os.walk(basepath):
        for filename in filenames:
            if filename.endswith('.zip'):
                tmppath=os.sep.join([dirpath, filename])
                with ZipFile(tmppath, 'r') as zipObj:
                   # Get a list of all archived file names from the zip
                   listOfFileNames = zipObj.namelist()
                   # Iterate over the file names
                   for fileName in listOfFileNames:
                       # Check filename endswith txt
                        if fileName.endswith('04.txt'):
                            zipObj.extract(fileName, os.path.join(basepath,'temp_txt'))
                        if fileName.endswith('09.txt'):
                            zipObj.extract(fileName, os.path.join(basepath,'temp_txt'))
infile = os.path.join(basepath,'temp_txt')

In [154]:
a=HEI.file_org(infile, arglist, important)

this is the item Chil
Chil
Chil
Chil
Final dataframe size:  (931, 226)


In [155]:
df=a[arglist['OPTS'][0]]

In [156]:
df.shape

(659, 382)

In [157]:
def path_finder(arglist):
    #read in the data
    df1=os.path.join(arglist['BASEPATH'],arglist['SAVE'],arglist['XTRA'])
    print(df1)
    demo_df=pd.read_csv(df1, sep=',',encoding='latin1')
    return(demo_df)

In [158]:
def refactor(infant_df):
    infant_df['breastfed']=infant_df['breastfed'].replace({'no': 0, 'yes': 1})
    infant_df['any_formula']=infant_df['any_formula'].replace({'no': 0, 'yes': 1,'NaN':'NA'})
    infant_df['regular_formula']=infant_df['regular_formula'].replace({'no': 0, 'yes': 1,'NaN':'NA','not_answered':'NA'})
    infant_df['age_fed_dropdown']=infant_df['age_fed_dropdown'].replace({'no': 0, 'yes': 1,'NaN':'NA','not_answered':'NA','never_not_yet':0})
    infant_df['age_stop_dropdown']=infant_df['age_stop_dropdown'].replace({'no': 0, 'yes': 1,'NaN':'NA','not_answered':'NA','never_not_yet':0})
    return(infant_df)

In [188]:
def ager(x):
    # Get the age in months of each recall
    x['DoB'] = pd.to_datetime(x['DoB'])
    x['Date of Intake'] = pd.to_datetime(x['Date of Intake'])
    x['Date_taken'] = pd.to_datetime(x['Date_taken'])
    
    x['Age_at_Date_of_Intake'] = x['Date of Intake'] - x['DoB']
    x['Age_at_Date_of_Anthros'] = x['Date_taken'] - x['DoB']
    
    x['Age_at_Date_of_Intake']= x['Age_at_Date_of_Intake']/ np.timedelta64(1, 'M')
    x['Age_at_Date_of_Anthros']= x['Age_at_Date_of_Anthros']/ np.timedelta64(1, 'M')
    return(x)

## Important Feeding stuff
* DoB
* Started Any Formula (Months old) NA - No Formula
* Started Regular (in months)
* Ended Formula (months old) NA - No Formula
* Started Solids (months old)
* Ended breastmilk (months old)
* Feeding Practice (First 4 months)
* child_feeding_practice-age_any_formula
* child_feeding_practice-age_fed
* child_feeding_practice-age_fed_dropdown
* child_feeding_practice-age_fed_status
* child_feeding_practice-age_regular_formula
* child_feeding_practice-age_regular_formula_status
* child_feeding_practice-age_stop
* child_feeding_practice-age_stop_dropdown
* child_feeding_practice-age_stop_formula
* child_feeding_practice-age_stop_formula_dropdown
* child_feeding_practice-age_stop_formula_status
* child_feeding_practice-age_stop_status
* child_feeding_practice-any_formula
* child_feeding_practice-breastfed
* child_feeding_practice-regular_formula
* Date_taken
* Date of Intake

In [189]:
def BCP(diet_df, arglist):
    diet_df=df
    demo_df = path_finder(arglist)
    y='Participant ID'
    diet_df[y]=diet_df[y].convert_objects(convert_numeric=True)
    demo_df[y]=demo_df[y].convert_objects(convert_numeric=True)
    unique1 = list(set(demo_df[y])-set(diet_df[y])) 
    print(unique1)
    unique2 = list(set(diet_df[y])-set(demo_df[y])) 
    print(unique2)
    all_data=demo_df.merge(diet_df, on=[y])
    all_data=ager(all_data)

    return(all_data)

In [190]:
x=BCP(df, arglist)

/Users/gracer/Google Drive/BCP/data/derivived/total_dataset.csv
[914176, 476291, 947206, 657158, 999947, 780046, 759162, 437532, 251549, 866461, 714783, 295330, 236452, 682023, 569511, 640680, 536876, 387450, 298032, 943541, 861367, 232633, 469822, 764612, 904006, 318409, 633038, 427732, 710741, 680534, 209364, 617942, 871514, 474330, 375518, 736609, 865762, 709477, 705639, 351080, 317415, 309615, 863856, 124529, 513010, 671741, 826360, 456442, 747645, 887679]
[107008, 525955, 17541, 406663, 683402, 151307, 223244, 530066, 740371, 607764, 216852, 157075, 266394, 200474, 385434, 2081, 757282, 624802, 494244, 788132, 605985, 111521, 408997, 412584, 395568, 158643, 137012, 505525, 761141, 110006, 340476, 950202, 148796, 417855, 106436, 189896, 456907, 758603, 51407, 382287, 388560, 732627, 911829, 415958, 627929, 468570, 859865, 509659, 945369, 314462, 999647, 675674, 546401, 11228, 354404, 648807, 452200, 67688, 418793, 495082, 865900, 801901, 576112, 515697, 52977, 774387, 683764, 42110

In [191]:
y=HEI.make_ped_components(hei_ped_dict, x, conv_dict)

hei_totveg
hei_greensbeans
hei_totfruit
hei_wholefruit
hei_wholegrains
hei_dairy
hei_totproteins
hei_seafoodplantprot
hei_refinedgrains
energy
hei_fruitjuice
hei_SSB
chocolate_candies
candies
frosting
sweet_sauce
sugar
syrups
Pudding
icecream
nondairy_treat
baked_good
chips
fries
other_fried
milk
formula_foz
cereal_oz
bbcereal_hcup


In [192]:
que=HEI.make_hei(y, make_hei_dict)


START
{'hei_sweets': ['chocolate_candies', 'candies', 'frosting', 'sweet_sauce', 'sugar', 'syrups', 'Pudding', 'icecream', 'nondairy_treat', 'baked_good'], 'hei_salty': ['chips', 'other_fried', 'fries'], 'hei_vegetables': ['hei_totveg', 'hei_greensbeans'], 'hei_proteins': ['hei_totproteins', 'hei_seafoodplantprot'], 'hei_milk': ['hei_dairy', 'formula_foz'], 'hei_cereal': ['cereal_oz', 'bbcereal_hcup']}


In [241]:
def splitter(DF):
    DF_child=DF.query('Age_at_Date_of_Intake >= 12')
    DF_young=DF.query('Age_at_Date_of_Intake < 12 and Age_at_Date_of_Intake >= 8')
    DF_infant=DF.query('Age_at_Date_of_Intake < 8')
    DF_infant['start_regular_formula']=DF_infant.iloc[:,[15]]
    DF_infant['start_regular_formula']=DF_infant['start_regular_formula'].astype('float32')
    df={'DF_child':DF_child, 'DF_young':DF_young, 'DF_infant':DF_infant}
    return(df)

In [242]:
split_dict=splitter(que)

In [243]:
split_dict['DF_young']['Age_at_Date_of_Intake'].describe()

count    21234.000000
mean        10.073972
std          1.072412
min          8.115156
25%          9.232222
50%          9.823610
75%         10.940676
max         11.992033
Name: Age_at_Date_of_Intake, dtype: float64

In [244]:
split_dict['DF_infant']['Started Any Formula (Months old) NA - No Formula'].value_counts()

0.0     2936
1       1680
6.0     1290
11.0    1176
0.4     1096
0.13     840
0.1      820
10       760
0.4      720
7.0      672
7        630
10.0     588
4.5      480
0.16     392
8.0      360
0.5      336
1.4      336
5        144
3        120
9         90
Name: Started Any Formula (Months old) NA - No Formula, dtype: int64

In [263]:
def DQI_BF(df, output, age_group):
    if age_group == 'infant':
        df[output]=[15 if pd.isna(row['start_regular_formula']) == True else 5 if row['start_regular_formula'] >= 8 else 10 for index, row in df.iterrows()]
    else:
        df[output]=[10 if row['age_stop_dropdown'] == 0 and row['breastfed'] == 1 else 0 for index, row in df.iterrows()]
        
        
        
        
        

In [264]:
def check(dic, data, name, option, arglist):
    if arglist['CHILD'] == False:
        df=data
        toSum=['HEIX1_TOTALVEG','HEIX2_GREEN_AND_BEAN' , 'HEIX3_TOTALFRUIT' , 'HEIX4_WHOLEFRUIT' ,
               'HEIX5_WHOLEGRAIN' , 'HEIX6_TOTALDAIRY' , 'HEIX7_TOTPROT' , 'HEIX8_SEAPLANT_PROT' , 'HEIX9_FATTYACID' ,
               'HEIX10_SODIUM' , 'HEIX11_REFINEDGRAIN' , 'HEIX12_ADDEDSUGARS' , 'HEIX13_SATFATS']
        for key,values in dic.items():
            print('Calculating score for %s'%key)
            if key in ['hei_totveg','hei_greensbeans', 'hei_totfruit', 'hei_wholefruit', 'hei_totproteins', 'hei_seafoodplantprot',
                        'hei_wholegrains', 'hei_dairy','Fats']:
                if key == 'Fats':
                    adeq_check(df, key, values['name'], values['parameters'])
                else:
                    adeq_check(df, key, values['name'], values['parameters'][0])
            if key in ['hei_sodium','hei_refinedgrains','hei_addedsugars','hei_SFA']:
                mod_check(df, key, values['name'], values['parameters'])
        df['HEI2015_TOTAL_SCORE']=df[df.columns.intersection(toSum)].sum(axis=1)
        concat_filepath = os.path.join(arglist['SAVE'],'%s_%s_HEI.csv'%(option, name))
        df.to_csv(concat_filepath, index=False, sep=",", header=True)
    else:
        df=data
        for key,values in dic.items():
            if key in ['hei_vegetables','hei_totfruit','hei_wholegrains','hei_dairy','hei_milk','hei_proteins','hei_refinedgrains',
            'hei_fruitjuice','hei_SSB','hei_sweets','hei_salty','hei_cereal']:
                print('Calculating score for %s'%key)
                if name != 'infant':
                    toSum=['HEIX0_BREASTFEEDING','HEIX1_VEGETABLES','HEIX2_TOTALFRUIT' , 'HEIX3_WHOLEGRAIN' , 'HEIX4_TOTALDAIRY' ,
                           'HEIX5_PROTEIN' , 'HEIX6_REFINEDGRAIN' , 'HEIX7_FRUITJUICE' , 'HEIX8_SSB', 'HEIX9_SWEETS',
                           'HEIX10_SALTY']
                    DQI(df,key, values['name'], values['parameters'])
                    df['HEI2015_TOTAL_SCORE']=df[df.columns.intersection(toSum)].sum(axis=1)
                    concat_filepath = os.path.join(arglist['SAVE'],'%s_%s_HEI.csv'%(option, name))
                    df.to_csv(concat_filepath, index=False, sep=",", header=True)
                else:
                    toSum=['HEIX0_BREASTFEEDING','HEIX1_VEGETABLES','HEIX2_TOTALFRUIT'  ,
                           'HEIX5_PROTEIN' ,  'HEIX7_FRUITJUICE' , 'HEIX8_SSB', 'HEIX9_SWEETS',
                           'HEIX10_SALTY', 'HEIX11_CEREAL']
                    infant_DQI(df,key, values['name'], values['parameters'])
                    df['HEI2015_TOTAL_SCORE']=df[df.columns.intersection(toSum)].sum(axis=1)
                    concat_filepath = os.path.join(arglist['SAVE'],'%s_%s_HEI.csv'%(option, name))
                    df.to_csv(concat_filepath, index=False, sep=",", header=True)


In [265]:
DQI_BF(split_dict['DF_infant'], 'HEIX0_BREASTFEEDING', 'infant')

In [267]:
split_dict['DF_infant']['HEIX0_BREASTFEEDING'].describe()

count    25460.000000
mean        12.668892
std          3.122961
min          5.000000
25%         10.000000
50%         15.000000
75%         15.000000
max         15.000000
Name: HEIX0_BREASTFEEDING, dtype: float64

In [252]:
split_dict['DF_infant']['start_regular_formula'].describe()

count    10072.000000
mean         4.541829
std          3.388052
min          0.000000
25%          0.500000
50%          6.000000
75%          6.500000
max         10.000000
Name: start_regular_formula, dtype: float64

In [202]:
for key, value in split_dict.items():
    if key == 'DF_child':
        HEI.DQI_BF(value, 'HEIX0_BREASTFEEDING', 'child')
        HEI.check(ped_dict['child'], value, 'child', key, arglist)
    elif key == 'DF_young':
        HEI.DQI_BF(value, 'HEIX0_BREASTFEEDING', 'young')
        HEI.check(ped_dict['young'], value, 'young', key, arglist)
    else:
        HEI.DQI_BF(value, 'HEIX0_BREASTFEEDING', 'infant')
        HEI.check(ped_dict['infant'], value, 'infant', key, arglist)

KeyError: 'age_stop_dropdown'

In [134]:
pd.DataFrame.to_csv(y, os.path.join(basepath,'derivived','total_dataset.csv'), sep=',')